# Data Prepration

## Feature engineering for app labels

#### Here we will do another feature engineering and extract all the labels of a app per device id

In [166]:
# importing libraries
import pandas as pd
import numpy as np

In [167]:
#Load this files from S3
df_labels =  pd.read_csv('https://upgradcapstone.s3.amazonaws.com/app-data/app_labels.csv')
df_label_cat =  pd.read_csv('https://upgradcapstone.s3.amazonaws.com/app-data/label_categories.csv')
df_app_events = pd.read_csv('https://upgradcapstone.s3.amazonaws.com/app-data/app_events.csv')
df_test_train = pd.read_csv('https://upgradcapstone.s3.amazonaws.com/train-test-split/train_test_split.csv')
#df_labels

In [168]:
# data from previous notebooks
df_event =  pd.read_csv('event_data.csv')
df_non_event =  pd.read_csv('non_event_data.csv')

C:\Users\chand\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\chand\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (9,10,11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Merge lable and categories dataset

In [169]:
df_join_lable_cat = df_labels.merge(df_label_cat, how='left', on='label_id')
df_join_lable_cat

,app_id,label_id,category
0,7324884708820027918,251,Finance
1,-4494216993218550286,251,Finance
2,6058196446775239644,406,unknown
3,6058196446775239644,407,DS_P2P net loan
4,8694625920731541625,406,unknown
...,...,...,...
459938,8899923330228547048,932,travel
459939,8906927134640865376,932,travel
459940,9180397886043162136,932,travel
459941,9204453858293451002,932,travel


In [170]:
df_join_lable_cat.category.value_counts()
#df_app_labels.groupby('category')['category'].count()

Industry tag             56902
Custom label             53936
Tencent                  49320
game                     48707
Property Industry 2.0    45697
                         ...  
Intelligent hardware         1
poker and chess              1
Children puzzle game         1
game-Gem Elimination         1
psychology                   1
Name: category, Length: 473, dtype: int64

Here we calulcate all the labels belonging to a particular app_id

In [171]:
df_app_labels = df_join_lable_cat.groupby('app_id').label_id.apply(lambda x: ' '.join(str(s) for s in x))
df_app_labels.head()

app_id
-9223281467940916832                796 795 794 405
-9222877069545393219                            135
-9222785464897897681                812 795 794 405
-9222198347540756780                810 795 794 405
-9221970424041518544    714 704 548 813 795 794 405
Name: label_id, dtype: object

In [172]:
# with the app abd its label data, join this with app events data
df_app_events ['app_label'] = df_app_events['app_id'].map(df_app_labels)

In [173]:
df_app_events.head()

,event_id,app_id,is_installed,is_active,app_label
0,2,5927333115845830913,1,1,549 710 704 548 172
1,2,-5720078949152207372,1,0,721 704 548 302 303
2,2,-1633887856876571208,1,0,251 263 306 302 405 730 756 757 775 779 783 1007
3,2,-653184325010919369,1,1,405 730 747 749 776 782 785 255 251 1014
4,2,8693964245073640147,1,1,549 710 704 548 172


Now lts calulate all the labels that belong to event ids

In [174]:
df_event_labels = df_app_events.groupby('event_id').app_label.apply(lambda x: ' '.join(str(s) for s in x))
df_event_labels.tail()

event_id
3252935    549 713 704 548 237 168 713 704 548 549 710 70...
3252936              713 704 548 549 710 704 548 172 302 303
3252937    706 704 548 178 549 721 704 302 303 548 183 54...
3252946    1012 706 704 548 548 549 549 721 704 302 303 5...
3252948    549 713 704 548 549 715 704 306 302 548 167 54...
Name: app_label, dtype: object

In [175]:
df_event['app_labels'] = df_event.event_id.map(df_event_labels)
df_event.app_labels = df_event.app_labels.fillna('Missing')
df_event.tail()

,device_id,gender,age,group,event_id,timestamp,longitude,latitude,has_event,week_day,hour_day,week,year,group_cat,hour_day_cat,loc_cluster,app_labels
1215593,4521177520949810000,F,26,F25-32,1458305.0,2016-05-04 18:44:30,0.0,0.0,1,Wednesday,18,18,2016,25-32,Afternoon,Unknown,548 549 711 714 548 704 813 795 794 405 27 549...
1215594,4521177520949810000,F,26,F25-32,504315.0,2016-05-05 09:43:15,0.0,0.0,1,Thursday,9,18,2016,25-32,Morning,Unknown,548 549 711 714 548 704 813 795 794 405 27 549...
1215595,7532333004539760000,M,29,M25-32,511683.0,2016-05-02 18:02:21,0.0,0.0,1,Monday,18,18,2016,25-32,Afternoon,Unknown,549 721 704 302 303 548 183 549 710 704 548 17...
1215596,7532333004539760000,M,29,M25-32,1725229.0,2016-05-02 18:07:45,0.0,0.0,1,Monday,18,18,2016,25-32,Afternoon,Unknown,549 721 704 302 303 548 183 549 710 704 548 17...
1215597,7532333004539760000,M,29,M25-32,1545030.0,2016-05-02 18:02:37,0.0,0.0,1,Monday,18,18,2016,25-32,Afternoon,Unknown,549 721 704 302 303 548 183 549 710 704 548 17...


In [176]:
df_deviceId_labels = df_event.groupby('device_id').app_labels.apply(lambda x: ' '.join(str(s) for s in x))
df_deviceId_labels.head()

device_id
-9222956879900150000    959 960 548 1007 959 960 548 1007 959 960 548 ...
-9221026417907250000    Missing 549 710 704 548 172 721 704 548 302 30...
-9220830859283100000    710 704 548 549 710 704 548 172 721 704 548 30...
-9220061629197650000    Missing Missing Missing 1012 549 723 704 302 3...
-9218960997324660000    1012 713 704 548 302 303 163 549 710 704 548 1...
Name: app_labels, dtype: object

In [177]:
# collect the app lables against each device id
df_non_event['app_lables']= df_non_event['device_id'].map(df_deviceId_labels)
df_non_event.head(2)

,device_id,gender,age,group,phone_brand,device_model,group_cat,avg_events,most_active_hour,loc_chng_morning,loc_chng_afternoon,loc_chng_evening,loc_chng_night,loc_cluster,app_lables
0,-7548291590301750000,M,33,M32+,Huawei,è£è€€3C,33-45,58.4,15.0,True,True,True,True,0,Missing Missing Missing Missing Missing Missin...
1,-7548291590301750000,M,33,M32+,Huawei,è£è€€3C,33-45,58.4,15.0,True,True,True,True,0,Missing Missing Missing Missing Missing Missin...


In [178]:
# here we can get rid of unused datasets
del df_join_lable_cat
del df_app_labels
del df_event_labels
del df_deviceId_labels

In [179]:
#drop all the duplicates
df_non_event.drop_duplicates(subset=['device_id'],inplace=True)

In [180]:
df_non_event.head(2)

,device_id,gender,age,group,phone_brand,device_model,group_cat,avg_events,most_active_hour,loc_chng_morning,loc_chng_afternoon,loc_chng_evening,loc_chng_night,loc_cluster,app_lables
0,-7548291590301750000,M,33,M32+,Huawei,è£è€€3C,33-45,58.4,15.0,True,True,True,True,0,Missing Missing Missing Missing Missing Missin...
292,6943568600617760000,M,37,M32+,Xiaomi,xnote,33-45,1.0,11.0,False,False,False,False,Unknown,713 704 548 549 710 704 548 172 179 179 548 54...


## Convert the features in One-Hot-Encoded features

In [181]:
from sklearn.feature_extraction.text import CountVectorizer

In [182]:
vectorizer = CountVectorizer(binary=True)
df_app_lab_vectorized = vectorizer.fit_transform(df_non_event['app_lables'].fillna('Missing')) 
#str(vectorizer.get_feature_names())

In [183]:
df_app_vects = pd.DataFrame(df_app_lab_vectorized.toarray(), columns=vectorizer.get_feature_names(), index=df_non_event.device_id)
df_app_vects.head(3)

C:\Users\chand\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,10,100,1002,1003,1005,1006,1007,1008,1009,101,...,99,990,991,993,995,996,997,998,999,missing
device_id,,,,,,,,,,,,,,,,,,,,,
-7548291590301750000,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6943568600617760000,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5441349705980020000,0,1,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [184]:
df_non_event = df_non_event.merge(df_app_vects, how='left', left_on='device_id', right_index=True)
df_non_event.head(2)


,device_id,gender,age,group,phone_brand,device_model,group_cat,avg_events,most_active_hour,loc_chng_morning,...,99,990,991,993,995,996,997,998,999,missing
0,-7548291590301750000,M,33,M32+,Huawei,è£è€€3C,33-45,58.4,15.0,True,...,0,0,0,0,0,0,0,0,0,1
292,6943568600617760000,M,37,M32+,Xiaomi,xnote,33-45,1.0,11.0,False,...,0,0,0,0,0,0,0,0,0,0


In [185]:
# updated boolean values to int
df_non_event.loc_chng_morning = df_non_event.loc_chng_morning.map(lambda x: 1 if x==1 else 0)
df_non_event.loc_chng_afternoon = df_non_event.loc_chng_afternoon.map(lambda x: 1 if x==1 else 0)
df_non_event.loc_chng_evening = df_non_event.loc_chng_evening.map(lambda x: 1 if x==1 else 0)
df_non_event.loc_chng_night = df_non_event.loc_chng_night.map(lambda x: 1 if x==1 else 0)

In [186]:
df_non_event.drop(columns=['gender', 'age', 'group', 'app_lables'], inplace=True)

In [188]:
dataset_columns = df_non_event.columns
len(dataset_columns)

491

### Now save the test and train split in a separate file for further processing

In [189]:
df_train = df_test_train[df_test_train.train_test_flag== 'train']
df_test = df_test_train[df_test_train.train_test_flag== 'test']

In [191]:
dataset_columns[:20]

Index(['device_id', 'phone_brand', 'device_model', 'group_cat', 'avg_events',
       'most_active_hour', 'loc_chng_morning', 'loc_chng_afternoon',
       'loc_chng_evening', 'loc_chng_night', 'loc_cluster', '10', '100',
       '1002', '1003', '1005', '1006', '1007', '1008', '1009'],
      dtype='object')

In [192]:
# df with phone data phone related columns  and event & app related columns (ALL coumns)
df_scenerio_1 = df_non_event
# df with only phone related columns 
df_scenerio_2 = df_non_event[['device_id', 'phone_brand', 'device_model',]]

In [194]:
len(df_scenerio_1.columns)

491

Get dummy variables for the secnerio 1 and 2 data set

In [195]:
df_scenerio_1 = pd.get_dummies(df_scenerio_1,columns=['phone_brand', 'device_model', 'group_cat', 'loc_cluster'])
df_scenerio_1.shape #(74645, 4858)

(74645, 4858)

In [196]:
df_scenerio_2 = pd.get_dummies(df_scenerio_2,columns=['phone_brand', 'device_model'])
df_scenerio_2.shape #(74645, 4858)

(74645, 1536)

In [200]:
df_scenario1_train_data = df_train.merge(df_scenerio_1, how='left', on='device_id')
df_scenario1_test_data = df_test.merge(df_scenerio_1, how='left', on='device_id')

In [201]:
df_scenario2_train_data = df_train.merge(df_scenerio_2, how='left', on='device_id')
df_scenario2_test_data = df_test.merge(df_scenerio_2, how='left', on='device_id')

In [206]:
# drop train test flag clumn
df_scenario1_train_data.drop(columns=['train_test_flag'], inplace=True)
df_scenario1_test_data.drop(columns=['train_test_flag'], inplace=True)

df_scenario2_train_data.drop(columns=['train_test_flag'], inplace=True)
df_scenario2_test_data.drop(columns=['train_test_flag'], inplace=True)

In [216]:
df_scenario1_train_data.shape,df_scenario1_test_data.shape #((58549, 4861), (16096, 4861))

((58549, 4861), (16096, 4861))

In [217]:
df_scenario2_train_data.shape,df_scenario2_test_data.shape #((58549, 1539), (16096, 1539))

((58549, 1539), (16096, 1539))

### Save the **Scenerio 1** and **Secenrio 2** splits in individual files.

In [218]:
#save the dataset for next notebook
#scenerio 1
df_scenario1_train_data.to_csv('../5_Model_Building/scenario1_train.csv',index=False)
df_scenario1_test_data.to_csv('../5_Model_Building/scenario1_test.csv',index=False)

#scenerio 2
df_scenario2_train_data.to_csv('../5_Model_Building/scenario2_train.csv',index=False)
df_scenario2_test_data.to_csv('../5_Model_Building/scenario2_test.csv',index=False)

In [219]:
#delete the dataframes
del df_scenario1_train_data
del df_scenario1_test_data
del df_scenario2_train_data
del df_scenario2_test_data
del df_train
del df_test
del df_non_event
del df_labels 
del df_label_cat
del df_app_events
del df_test_train

---


End of this notebook